In [1]:
import jieba
import os
import re
import argparse
import pickle
import pandas as pd
from gensim import corpora, models, similarities
from utils import *
import logging
# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# * Load input data.
def load_documents(input_path):
    """Load input data"""
    with open(input_path, encoding="utf-8") as f:
        docs = f.readlines()
    return docs

In [2]:
docs = load_documents("data/training_set.txt")
docs = docs[:5]

In [4]:
print(docs[0])

黄蜂vs湖人首发：科比带伤战保罗 加索尔救赎之战 新浪体育讯北京时间4月27日，NBA季后赛首轮洛杉矶湖人主场迎战新奥尔良黄蜂，此前的比赛中，双方战成2-2平，因此本场比赛对于两支球队来说都非常重要，赛前双方也公布了首发阵容：湖人队：费舍尔、科比、阿泰斯特、加索尔、拜纳姆黄蜂队：保罗、贝里内利、阿里扎、兰德里、奥卡福[新浪NBA官方微博][新浪NBA湖人新闻动态微博][新浪NBA专题][黄蜂vs湖人图文直播室](新浪体育)



In [5]:
# * Pre-process that data.
def pre_process(docs, stopwords_path):
    """Pre-process that data"""
    stopwords = stopwords_list(stopwords_path)

    new_docs = []
    # 给每个文档分词
    for doc in docs:
        doc = re.sub(r'[^\u4e00-\u9fa5]+','', doc)
        doc_token = tokenize(doc.strip(), stopwords)
        new_docs.append(doc_token)
    
    train_docs = []
    for doc in new_docs:
        train_doc = [word.strip() for word in doc.split()]
        train_docs.append(train_doc)

    return train_docs

In [6]:
docs = pre_process(docs, "data/stopwords.txt")

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.737 seconds.
Prefix dict has been built successfully.


In [8]:
print(docs[0])

['黄蜂', '湖人', '首发', '科比', '带伤', '战', '保罗', '加索尔', '救赎', '之战', '新浪', '体育讯', '北京', '时间', '季后赛', '首轮', '洛杉矶', '湖人', '主场', '迎战', '新奥尔良', '黄蜂', '此前', '比赛', '中', '战成', '平', '本场', '比赛', '两支', '球队', '赛前', '公布', '首发', '阵容', '湖人队', '费舍尔', '科比', '阿泰斯特', '加索尔', '拜纳姆', '黄蜂队', '保罗', '贝里', '内利', '阿里', '扎', '兰德', '里', '奥卡福', '新浪', '官方', '微博', '新浪', '湖人', '新闻动态', '微博', '新浪', '专题', '黄蜂', '湖人', '图文', '直播室', '新浪', '体育']


In [9]:
# * Transform documents into bag-of-words vectors.
# * Construct dictionary.
def construct_dictionary(docs):
    """Construct word frequency dictionary"""
    dictionary = corpora.Dictionary(docs)
    return dictionary

In [10]:
dictionary = construct_dictionary(docs)

In [13]:
print(dictionary)

Dictionary(604 unique tokens: ['专题', '两支', '中', '主场', '之战']...)
